In [ ]:
import os

# Activate environment manually
# conda activate blueleaflabs

# Local project root
PROJECT_DIR = os.path.expanduser("~/hydropulse")
os.makedirs(os.path.join(PROJECT_DIR, "results", "manual"), exist_ok=True)
os.chdir(PROJECT_DIR)

In [ ]:
import os, json, math, io, zipfile, time, re, shutil, glob, pathlib
from datetime import datetime as dt, timedelta, timezone, date
from dateutil import parser as dateparser
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import requests
from urllib.parse import urljoin, quote
from bs4 import BeautifulSoup
from tqdm import tqdm
import pytz

import xarray as xr
import rasterio
from rasterio.mask import mask
import shapely
from shapely import ops 
from shapely.geometry import Point, Polygon, box, mapping
from shapely.ops import unary_union, transform as shp_transform
from pyproj import Transformer
import geopandas as gpd

import matplotlib.pyplot as plt
import folium
import ee  # Earth Engine

import warnings
warnings.filterwarnings("ignore")

print("Environment ready in VS Code.")

In [ ]:
# --- Configurations for local VS Code setup (California full extent) ---

PURPLEAIR_SENSOR_INDEX = ''  # optional manual override
CDS_UID = ''
CDS_API_KEY = ''

CONFIG = {
    # Project identity
    "parent": "blueleaflabs",
    "project": "hydropulse",

    # California bounding box (approximate)
    "bbox": {
        "nwlat": 42.0095,
        "nwlng": -124.4820,
        "selat": 32.5343,
        "selng": -114.1315
    },

    # Time window (UTC)
    "start_date": "2024-06-01",
    "end_date": "2024-10-31",

    # API tokens
    "CDO_TOKEN": "BrboArtFBPlkPVXhDLlmrUZiTLVwThSr",
    "AIRNOW_API_KEY": "BDF7B01F-7934-45E9-ABF7-5DC35F4E716C",
    "PURPLEAIR_API_KEY": "A2F84F7C-B439-11F0-BDE5-4201AC1DC121", #changed because the prior one ran out!
    "ACS_KEY": "b7e5e9c956393182b93ae8c0786895acdbbbf8e9",

    # Base URLs
    "USCRN_BASE_HOURLY": "https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/",
    "HRRR_BASE_GENERIC": "https://nomads.ncep.noaa.gov/cgi-bin/filter_hrrr_2d.pl",
    "HRRR_BASE_SMOKE":   "https://nomads.ncep.noaa.gov/cgi-bin/filter_hrrr_smoke_2d.pl",
    "HMS_BASE_SMOKE": "https://satepsanone.nesdis.noaa.gov/pub/FIRE/web/HMS/Smoke_Polygons/Shapefile",
    "HMS_BASE_FIRE":  "https://satepsanone.nesdis.noaa.gov/pub/FIRE/web/HMS/Fire_Points/Shapefile",
    "ACS_BASE_ACS5": "https://api.census.gov/data/2023/acs/acs5",
    "ACS_BASE_SUBJECT": "https://api.census.gov/data/2023/acs/acs5/subject",
    "WHP_ZIP": "https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/RDS-2015-0047-4_Data.zip",
    "WHP_TIF_NAME": "Data/whp2023_GeoTIF/whp2023_cnt_conus.tif",

    # Headers
    "USER_AGENT_HEADERS": {"User-Agent": "HeatShield/1.0"},
    "NOMADS_HEADERS": {"User-Agent": "HeatShield/1.0", "Referer": "https://nomads.ncep.noaa.gov/"},

    # HRRR
    "HRRR_CYCLES": ["00"],
    "HRRR_FCST_HOURS": [0],

    # Output paths (will be absolutized below)
    "out_dir": "results",
    "NLCD_MANUAL_ZIP": "results/manual/Annual_NLCD_LndCov_2024_CU_C1V1.zip",

    # Spatial config
    "grid_resolution_m": 3000, # grid cell size in meters; 500 was too small for all of CA!
    "crs_epsg": 4326,

    # USCRN configuration for California
    "USCRN_STATION_NAME": [
        "CA Bodega 6 WSW",
        "CA Fallbrook 5 NE",
        "CA Merced 23 WSW",
        "CA Redding 12 WNW",
        "CA Santa Barbara 11 W",
        "CA Stovepipe Wells 1 SW",
        "CA Yosemite Village 12 W"
    ],
    "USCRN_YEARS": list(range(2024, 2025)),

    # AQS parameters
    "AQS_STATE_CODE": "06",  # California
    "AQS_EMAIL": "rohan3142@gmail.com",
    "AQS_KEY": "berrymouse51"
}

# --- Local file structure ---
# TO DO: PROJECT_DIR is hardcoded, make it generic for anybody to get the code from GitHub
PROJECT_DIR = "/Users/rohan/code/blueleaflabs/hydropulse"
os.makedirs(os.path.join(PROJECT_DIR, "results", "manual"), exist_ok=True)

# Absolutize output paths
CONFIG["out_dir"] = os.path.join(PROJECT_DIR, "results")
CONFIG["NLCD_MANUAL_ZIP"] = os.path.join(CONFIG["out_dir"], "manual", "Annual_NLCD_LndCov_2024_CU_C1V1.zip")

# Set working directory
os.chdir(PROJECT_DIR)

print("Config loaded and local directories ready.")